In [2]:
# Import the Pandas package
import pandas as pd
import numpy as np
import re 
import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings

In [16]:
def clean_text(string,replace='_'):
    regex = '[^A-Za-z0-9]+'
    str1 = re.sub(regex, replace, string)
    regex = '[^A-Za-z_]+'
    return re.sub(regex, '', str1)

def load_files(year):
    temp_df = pd.read_csv(file_str.format(year), skiprows = 1)
    column_list = list(temp_df.columns)#.to_list()
    if('Population' in column_list or 'population' in column_list):
        return temp_df.rename(columns = lambda col: clean_text(col.lower().strip()))
    else: 
        return pd.read_csv(file_str.format(year), skiprows = 1).rename(columns = lambda col: clean_text(col.lower().strip()))
    
    
def remove_digits(string):
    return re.sub('\d', '', string)


def replace_emdash(string):
    return string.replace('–','-',100)

def replace_all_special(string):
    return re.sub('[^A-Za-z0-9, ]+', '-', string)

def replace_msa(string):
    string= string.replace('Micro Area','')
    string= string.replace('Metro Area','')
    #string= string.replace('M.S.A','')
    return string.strip()

def split_name(string):
    split = string.split(',')
    city= split[0].strip()
    state = split[1].strip()
    
    city_split=city.split('-')
    state_split=state.split('-')
    return city_split[0]+', '+state_split[0]

def clean_string(col):
    return col.apply(remove_digits).apply(replace_emdash).apply(replace_msa).apply(replace_all_special).apply(split_name)#.apply(replace_all_special)


In [13]:
file_str = "data/demographics/poverty_census_{}.csv"
dfs = []
for  i in range(2010,2018):
    dfs.append(load_files(i))

In [15]:
dfs[1]

,id,geographic_area_name,total_estimate_population_for_whom_poverty_status_is_determined,total_margin_of_error_population_for_whom_poverty_status_is_determined,total_estimate_age_under__years,total_margin_of_error_age_under__years,total_estimate_age_under__years_related_children_under__years,total_margin_of_error_age_under__years_related_children_under__years,total_estimate_age__to__years,total_margin_of_error_age__to__years,...,percent_below_poverty_level_estimate_mean_income_deficit_for_unrelated_individuals_dollars_,percent_below_poverty_level_margin_of_error_mean_income_deficit_for_unrelated_individuals_dollars_,percent_below_poverty_level_estimate_worked_full_time_year_round_in_the_past__months,percent_below_poverty_level_margin_of_error_worked_full_time_year_round_in_the_past__months,percent_below_poverty_level_estimate_worked_less_than_full_time_year_round_in_the_past__months,percent_below_poverty_level_margin_of_error_worked_less_than_full_time_year_round_in_the_past__months,percent_below_poverty_level_estimate_did_not_work,percent_below_poverty_level_margin_of_error_did_not_work,percent_below_poverty_level_estimate_percent_imputed_poverty_status_for_individuals,percent_below_poverty_level_margin_of_error_percent_imputed_poverty_status_for_individuals
0,310M100US10140,"Aberdeen, WA Micro Area",69610,1394,15008,411,14763,488,42800,1333,...,(X),(X),2.9,4.5,33.4,10.9,48.9,9.1,(X),(X)
1,310M100US10180,"Abilene, TX Metro Area",151022,5792,37829,825,37579,845,91854,5379,...,(X),(X),0.8,1.0,59.4,8.3,47.4,7.0,(X),(X)
2,310M100US10300,"Adrian, MI Micro Area",93886,957,21291,567,21145,616,58055,895,...,(X),(X),4.7,3.4,44.8,10.7,32.9,9.1,(X),(X)
3,310M100US10380,"Aguadilla-Isabela-San Sebasti?n, PR Metro Area",301587,948,70482,1068,70122,1161,184854,1054,...,(X),(X),10.3,6.7,74.3,10.2,78.3,3.6,(X),(X)
4,310M100US10420,"Akron, OH Metro Area",684916,1925,151614,1080,150907,1214,435621,1859,...,(X),(X),3.4,1.5,52.1,4.0,43.7,3.3,(X),(X)
5,310M100US10460,"Alamogordo, NM Micro Area",63396,869,16033,717,16033,717,37740,1303,...,(X),(X),0.8,1.5,24.9,16.2,48.0,14.3,(X),(X)
6,310M100US10500,"Albany, GA Metro Area",157802,4127,42243,2007,41989,2010,96567,3072,...,(X),(X),9.9,5.5,51.5,13.2,62.3,7.4,(X),(X)
7,310M100US10540,"Albany-Lebanon, OR Micro Area",117310,682,27666,407,27370,520,70940,943,...,(X),(X),2.8,2.8,47.1,12.0,49.1,7.9,(X),(X)
8,310M100US10580,"Albany-Schenectady-Troy, NY Metro Area",840939,2569,179782,725,179354,800,542494,2661,...,(X),(X),3.3,1.2,36.6,4.1,30.9,3.1,(X),(X)
9,310M100US10700,"Albertville, AL Micro Area",93161,486,22886,355,22881,356,56515,618,...,(X),(X),2.7,3.3,40.6,20.0,39.3,8.7,(X),(X)


In [19]:
for i in range(len(dfs)):
    temp_df = dfs[i]
    temp_df = temp_df[temp_df['geographic_area_name'].str.contains("Area")]
    temp_df['geographic_area_name'] = clean_string(temp_df['geographic_area_name'])
    dfs[i] = temp_df

In [20]:
dfs[1]

,id,geographic_area_name,total_estimate_population_for_whom_poverty_status_is_determined,total_margin_of_error_population_for_whom_poverty_status_is_determined,total_estimate_age_under__years,total_margin_of_error_age_under__years,total_estimate_age_under__years_related_children_under__years,total_margin_of_error_age_under__years_related_children_under__years,total_estimate_age__to__years,total_margin_of_error_age__to__years,...,percent_below_poverty_level_estimate_mean_income_deficit_for_unrelated_individuals_dollars_,percent_below_poverty_level_margin_of_error_mean_income_deficit_for_unrelated_individuals_dollars_,percent_below_poverty_level_estimate_worked_full_time_year_round_in_the_past__months,percent_below_poverty_level_margin_of_error_worked_full_time_year_round_in_the_past__months,percent_below_poverty_level_estimate_worked_less_than_full_time_year_round_in_the_past__months,percent_below_poverty_level_margin_of_error_worked_less_than_full_time_year_round_in_the_past__months,percent_below_poverty_level_estimate_did_not_work,percent_below_poverty_level_margin_of_error_did_not_work,percent_below_poverty_level_estimate_percent_imputed_poverty_status_for_individuals,percent_below_poverty_level_margin_of_error_percent_imputed_poverty_status_for_individuals
0,310M100US10140,"Aberdeen, WA",69610,1394,15008,411,14763,488,42800,1333,...,(X),(X),2.9,4.5,33.4,10.9,48.9,9.1,(X),(X)
1,310M100US10180,"Abilene, TX",151022,5792,37829,825,37579,845,91854,5379,...,(X),(X),0.8,1.0,59.4,8.3,47.4,7.0,(X),(X)
2,310M100US10300,"Adrian, MI",93886,957,21291,567,21145,616,58055,895,...,(X),(X),4.7,3.4,44.8,10.7,32.9,9.1,(X),(X)
3,310M100US10380,"Aguadilla, PR",301587,948,70482,1068,70122,1161,184854,1054,...,(X),(X),10.3,6.7,74.3,10.2,78.3,3.6,(X),(X)
4,310M100US10420,"Akron, OH",684916,1925,151614,1080,150907,1214,435621,1859,...,(X),(X),3.4,1.5,52.1,4.0,43.7,3.3,(X),(X)
5,310M100US10460,"Alamogordo, NM",63396,869,16033,717,16033,717,37740,1303,...,(X),(X),0.8,1.5,24.9,16.2,48.0,14.3,(X),(X)
6,310M100US10500,"Albany, GA",157802,4127,42243,2007,41989,2010,96567,3072,...,(X),(X),9.9,5.5,51.5,13.2,62.3,7.4,(X),(X)
7,310M100US10540,"Albany, OR",117310,682,27666,407,27370,520,70940,943,...,(X),(X),2.8,2.8,47.1,12.0,49.1,7.9,(X),(X)
8,310M100US10580,"Albany, NY",840939,2569,179782,725,179354,800,542494,2661,...,(X),(X),3.3,1.2,36.6,4.1,30.9,3.1,(X),(X)
9,310M100US10700,"Albertville, AL",93161,486,22886,355,22881,356,56515,618,...,(X),(X),2.7,3.3,40.6,20.0,39.3,8.7,(X),(X)
